In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup.py

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import starry
from scipy.linalg import svd
from scipy.linalg import cho_factor, cho_solve

In [ ]:
starry.config.lazy = False
starry.config.quiet = True

Generate the data:

In [ ]:
# Instantiate a map
map = starry.Map(10, inc=60, lazy=False)
theta = np.linspace(0, 360, 1000, endpoint=False)
X = map.design_matrix(theta=theta)

In [ ]:
# Random coefficients
np.random.seed(0)
yerr = 1e-2
y = yerr * np.random.randn(map.Ny)

In [ ]:
# Generate a light curve
ferr = 1e-4
flux = X.dot(y) + ferr * np.random.randn(len(theta))

In [ ]:
plt.plot(theta, flux)
plt.xlabel("theta [deg]")
plt.ylabel("relative flux");

Full linear solve:

In [ ]:
mu = np.zeros(map.Ny)
L = (yerr ** 2) * np.eye(map.Ny)

In [ ]:
map.set_data(flux, C=ferr ** 2)
map.set_prior(mu=mu, L=L)
yhat, cho_ycov = map.solve(design_matrix=X)
cho_ycov = np.tril(cho_ycov)

SVD linear solve:

In [ ]:
# Pre-compute
U, s, VT = svd(X)
M = np.linalg.matrix_rank(X)
B = np.array(VT[:M]).T

# Something like this could work if we
# figure out the correct normalization.
"""
t = np.linspace(0, 2 * np.pi, 1000, endpoint=False).reshape(-1, 1)
W = np.hstack(
    [np.ones_like(t)]
    + [
        np.hstack([np.cos(n * t), np.sin(n * t)]) / (2 * n + 1) ** 2
        for n in range(1, map.ydeg + 1)
    ]
)
B = np.linalg.solve(W.T.dot(W), W.T.dot(X)).T
"""

# Solve
xhat, cho_xcov = starry.linalg.solve(
    X.dot(B), flux, C=ferr ** 2, mu=B.T.dot(mu), L=B.T.dot(L).dot(B)
)

# Transform the mean
yhat2 = B.dot(xhat)

# Transform the covariance
# NOTE: It's easy to get the inverse posterior covariance, but
# we need to invert things *twice* to get the cholesky
# factorization of the posterior covariance.
# Depending on the use case, we might want to avoid this.
invycov = B.dot(cho_solve((cho_xcov, True), B.T))
invycov += np.linalg.inv(L)
cho_invycov = cho_factor(invycov, lower=True)
ycov = cho_solve(cho_invycov, np.eye(L.shape[0]))
cho_ycov2, _ = cho_factor(ycov, lower=True)
cho_ycov2 = np.tril(cho_ycov2)

Compare:

In [ ]:
plt.plot(y, "k--", label="true")
plt.plot(yhat, lw=3, label="full")
plt.plot(yhat2, lw=1, label="svd")
std = np.sqrt(np.diag(cho_ycov.dot(cho_ycov.T)))
plt.fill_between(np.arange(len(yhat)), yhat - std, yhat + std, color="C0", alpha=0.3)
plt.legend(fontsize=10)
plt.xlabel("coefficient index")
plt.ylabel("value");

In [ ]:
np.allclose(yhat, yhat2)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8, 4))
ax[0].imshow(np.log10(np.abs(cho_ycov)), vmin=-15, vmax=0)
ax[1].imshow(np.log10(np.abs(cho_ycov2)), vmin=-15, vmax=0);

In [ ]:
plt.plot(np.diag(cho_ycov.dot(cho_ycov.T)), lw=3, label="full")
plt.plot(np.diag(cho_ycov2.dot(cho_ycov2.T)), lw=1, label="svd")
plt.legend(fontsize=10)
plt.xlabel("coefficient number")
plt.ylabel("uncertainty");

In [ ]:
np.allclose(cho_ycov, cho_ycov2, atol=1e-4)